### Ollama gpt-oss:20b 로드

python 3.12.3사용중

In [1]:
!nvidia-smi
!python -c "import torch,platform,sys; print('py',sys.version); print('cuda?',torch.cuda.is_available())"


Sun Sep  7 11:59:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.15              Driver Version: 570.86.15      CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-PCIE-40GB          Off |   00000000:8A:00.0 Off |                    0 |
| N/A   28C    P0             31W /  250W |       1MiB /  40960MiB |     N/A      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%###########                                                   32.8%
>>> Creating ollama user...
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [4]:
# GPU 환경변수 설정 후 서버 실행
!export OLLAMA_USE_GPU=1
# 선택: 특정 GPU만 쓰려면 (예: 0번 GPU만)
!export CUDA_VISIBLE_DEVICES=0

In [6]:
# ✅ Jupyter에서 Ollama 서버를 GPU 모드로 '비차단' 실행하기
import os, subprocess, time, signal, requests

# 1) 환경변수: GPU 사용
env = os.environ.copy()
env["OLLAMA_USE_GPU"] = "1"          # 서버 프로세스에 적용됨
env["CUDA_VISIBLE_DEVICES"] = "0"    # 필요시 GPU 인덱스 지정

# 2) 로그 파일
LOG = "/tmp/ollama.log"
logf = open(LOG, "ab", buffering=0)

# 3) 기존 떠 있는 ollama serve가 있으면 정리(선택)
try:
    subprocess.run(["pkill", "-f", "ollama serve"], check=False)
    time.sleep(1)
except Exception:
    pass

# 4) 비차단 실행 (주피터에선 & 금지 → Popen으로 대체)
#    preexec_fn=os.setpgrp 로 자식 프로세스 그룹을 분리(노트북 중지와 분리)
proc = subprocess.Popen(
    ["ollama", "serve"],
    stdout=logf, stderr=subprocess.STDOUT,
    env=env,
    preexec_fn=os.setpgrp  # 리눅스/WSL2
)

# 5) 헬스체크
time.sleep(2)
try:
    r = requests.get("http://127.0.0.1:11434/api/tags", timeout=3)
    print("✔️ Ollama up:", r.status_code, r.json())
except Exception as e:
    print("❌ 서버 확인 실패:", e)

# 6) 최근 로그 몇 줄 보기
try:
    with open(LOG, "rb") as f:
        tail = f.read().splitlines()[-20:]
    print("\n--- /tmp/ollama.log (tail) ---")
    for line in tail:
        try:
            print(line.decode("utf-8", "ignore"))
        except:
            print(line)
except FileNotFoundError:
    print("로그가 아직 생성되지 않았어요.")


✔️ Ollama up: 200 {'models': []}

--- /tmp/ollama.log (tail) ---
Couldn't find '/home/work/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIKg3rjQ0249Jy85HS79DUi3L3QlJN71c0Fv6CbmSj5Bl

time=2025-09-07T12:04:09.349+09:00 level=INFO source=routes.go:1331 msg="server config" env="map[CUDA_VISIBLE_DEVICES:0 GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/home/work/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NEW_ESTIMATES:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost h

#### ollama server 종료

In [37]:
import subprocess
subprocess.run(["pkill", "-f", "ollama serve"], check=False)

CompletedProcess(args=['pkill', '-f', 'ollama serve'], returncode=0)

#### ollama serve 특정 모델 위치에서 실행

In [38]:
env = os.environ.copy()
env["OLLAMA_MODELS"] = "./models"
subprocess.Popen(["ollama", "serve"], env=env)

<Popen: returncode: None args: ['ollama', 'serve']>

Couldn't find '/home/work/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIFaxEXINuJSeK/YZNJET7jm45Muu9reP5getxQ09UoXW



time=2025-09-07T12:35:43.035+09:00 level=INFO source=routes.go:1331 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:./models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NEW_ESTIMATES:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_SCHED_SPREAD:false

[GIN] 2025/09/07 - 12:35:48 | 200 |      49.897µs |       127.0.0.1 | HEAD     "/"
[GIN] 2025/09/07 - 12:35:48 | 200 |    2.577167ms |       127.0.0.1 | GET      "/api/tags"
[GIN] 2025/09/07 - 12:35:50 | 200 |      25.541µs |       127.0.0.1 | HEAD     "/"


time=2025-09-07T12:35:52.389+09:00 level=INFO source=download.go:177 msg="downloading aeda25e63ebd in 16 208 MB part(s)"
time=2025-09-07T12:36:00.143+09:00 level=INFO source=download.go:177 msg="downloading e0a42594d802 in 1 358 B part(s)"
time=2025-09-07T12:36:01.642+09:00 level=INFO source=download.go:177 msg="downloading dd084c7d92a3 in 1 8.4 KB part(s)"
time=2025-09-07T12:36:03.145+09:00 level=INFO source=download.go:177 msg="downloading 3116c5225075 in 1 77 B part(s)"
time=2025-09-07T12:36:04.629+09:00 level=INFO source=download.go:177 msg="downloading b6ae5839783f in 1 489 B part(s)"


[GIN] 2025/09/07 - 12:36:32 | 200 | 42.076637056s |       127.0.0.1 | POST     "/api/pull"


### ollama - gpt-oss:20b

In [13]:
import os, subprocess

# 원하는 경로 지정
os.environ["OLLAMA_MODELS"] = "./models"

# 디렉토리가 없으면 먼저 생성
os.makedirs(os.environ["OLLAMA_MODELS"], exist_ok=True)

# 모델 풀
subprocess.run(["ollama", "pull", "gpt-oss:20b"], check=True, env=os.environ)

[GIN] 2025/09/07 - 12:09:28 | 200 |      60.501µs |       127.0.0.1 | HEAD     "/"


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ 

[GIN] 2025/09/07 - 12:09:29 | 200 |   813.84471ms |       127.0.0.1 | POST     "/api/pull"


pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling b112e727c6f1: 100% ▕██████████████████▏  13 GB                         
pulling fa6710a93d78: 100% ▕██████████████████▏ 7.2 KB                         
pulling f60356777647: 100% ▕██████████████████▏  11 KB                         
pulling d8ba2f9a17b3: 100% ▕██████████████████▏   18 B                         
pulling 55c108d8e936: 100% ▕██████████████████▏  489 B                         
verifying sha256 digest 
writing manifest 
success 


CompletedProcess(args=['ollama', 'pull', 'gpt-oss:20b'], returncode=0)

In [20]:
!ollama list

[GIN] 2025/09/07 - 12:13:58 | 200 |      25.255µs |       127.0.0.1 | HEAD     "/"
[GIN] 2025/09/07 - 12:13:58 | 200 |      4.7796ms |       127.0.0.1 | GET      "/api/tags"
NAME           ID              SIZE     MODIFIED      
gpt-oss:20b    aa4295ac10c3    13 GB    4 minutes ago    


In [21]:
!pwd

/home/work/IROLIM/AIHumanities


In [26]:
!pip install langchain langchain-ollama

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/bitsandbytes-0.45.4.dev0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.12.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/nvfuser-0.2.23a0+6627725-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPR

In [15]:
import requests
resp = requests.post(
    "http://127.0.0.1:11434/api/generate",
    json={"model": "gpt-oss:20b", "prompt": "GPU로 추론 중인지 확인하는 간단한 방법을 한 문장으로 말해줘(한국어).", "stream": False},
    timeout=120,
)
print(resp.json()["response"])


time=2025-09-07T12:10:01.931+09:00 level=INFO source=server.go:199 msg="model wants flash attention"
time=2025-09-07T12:10:01.932+09:00 level=INFO source=server.go:216 msg="enabling flash attention"
time=2025-09-07T12:10:01.932+09:00 level=WARN source=server.go:224 msg="kv cache type not supported by model" type=""
time=2025-09-07T12:10:01.932+09:00 level=INFO source=server.go:398 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --model models/blobs/sha256-b112e727c6f18875636c56a779790a590d705aec9e1c0eb5a97d51fc2a778583 --port 40953"
time=2025-09-07T12:10:01.959+09:00 level=INFO source=runner.go:1251 msg="starting ollama engine"
time=2025-09-07T12:10:01.959+09:00 level=INFO source=runner.go:1286 msg="Server listening on 127.0.0.1:40953"
time=2025-09-07T12:10:02.235+09:00 level=INFO source=server.go:503 msg="system memory" total="1007.1 GiB" free="925.8 GiB" free_swap="8.0 GiB"
time=2025-09-07T12:10:02.520+09:00 level=INFO source=memory.go:36 msg="new model will f

[GIN] 2025/09/07 - 12:10:37 | 200 | 37.108818955s |       127.0.0.1 | POST     "/api/generate"
PyTorch를 사용한다면 `torch.cuda.is_available()`가 **True**이고, 모델의 파라미터가 `next(model.parameters()).device.type == 'cuda'`인지 한 줄로 확인하면 GPU로 추론 중인지 바로 판별할 수 있습니다.


In [27]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

# Ollama 연결
llm = ChatOllama(
    base_url="http://127.0.0.1:11434",  # ollama serve가 켜져 있는 주소
    model="gpt-oss:20b",
    temperature=0.7,
    num_ctx=2048,
)

time=2025-09-07T12:17:57.045+09:00 level=INFO source=server.go:199 msg="model wants flash attention"
time=2025-09-07T12:17:57.045+09:00 level=INFO source=server.go:216 msg="enabling flash attention"
time=2025-09-07T12:17:57.045+09:00 level=WARN source=server.go:224 msg="kv cache type not supported by model" type=""
time=2025-09-07T12:17:57.045+09:00 level=INFO source=server.go:398 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --model models/blobs/sha256-b112e727c6f18875636c56a779790a590d705aec9e1c0eb5a97d51fc2a778583 --port 44263"
time=2025-09-07T12:17:57.072+09:00 level=INFO source=runner.go:1251 msg="starting ollama engine"
time=2025-09-07T12:17:57.072+09:00 level=INFO source=runner.go:1286 msg="Server listening on 127.0.0.1:44263"
time=2025-09-07T12:17:57.299+09:00 level=INFO source=server.go:503 msg="system memory" total="1007.1 GiB" free="928.9 GiB" free_swap="8.0 GiB"
time=2025-09-07T12:17:57.549+09:00 level=INFO source=memory.go:36 msg="new model will f

[GIN] 2025/09/07 - 12:18:03 | 200 |  8.146732935s |       127.0.0.1 | POST     "/api/chat"
저는 LangChain과 Ollama를 통해 실행되는 AI 언어 모델로, 여러분의 질문에 답하고 도움을 드립니다.


In [29]:
# 단일 메시지
resp = llm.invoke("장곡사 미륵불 괘불탱에 대해서 설명해줘.")
print(resp.content)


[GIN] 2025/09/07 - 12:21:25 | 200 | 40.488654103s |       127.0.0.1 | POST     "/api/chat"
## 장곡사 미륵불 괘불탱이란?

### 1. 장곡사(장곡사)란?
- **위치**: 경상북도 경주(전통적으로 경주성 안에 위치한 작은 사찰)  
- **역사**: 조선시대에 설립되었으며, 사찰 자체는 규모가 작지만, 그 안에 보존된 미륵불은 한국 불교 문화의 보물로 평가됩니다.  
- **문화유산**: 장곡사는 ‘경주 사적지구’에 포함되어 유네스코 세계문화유산으로 지정된 경주 지역의 한 축을 담당합니다.

### 2. 미륵불 괘불탱(미륵불 괘불탱) 구조
| 항목 | 내용 |
|------|------|
| **형태** | 직사각형 구조의 소형 사당(‘탱’은 ‘탱’형 사당’을 뜻함) |
| **건축 시기** | 12세기 후반, 고려시대(1195년경) |
| **특징** | 기와(瓦) 지붕, 석판(石板) 기반, 내부는 한 개의 큰 동상만을 배치한 단순함이 특징 |
| **용도** | 미륵불을 모시고 예불·공덕을 쌓는 공간 |

### 3. 미륵불(미륵불) 동상
| 항목 | 내용 |
|------|------|
| **재료** | 청동(동) |
| **크기** | 약 4.5m (높이) |
| **무게** | 약 5톤 |
| **제작 시기** | 1184년(고려 왕경종 3년) |
| **제작자** | 알려지지 않음(전통적으로 ‘고려 청동조각가’라 전해짐) |
| **특징** | <br>1. **손 모양**: 오른손은 ‘무릉’ 모양(종교적 인형)으로 향해 있고, 왼손은 ‘삼지’ 모양으로 가라앉혀 있다.<br>2. **표정**: 고요하고 온화한 미소, 눈은 살짝 감겨 있는 형태.<br>3. **의복**: 금박이 발달한 장식이 돋보이는 긴 옷, 머리카락은 긴 물결 모양.<br>4. **꽃**: 오른손에 꽃(연꽃)을 들고 있는 모습. |
| **예술적 의의** | 고려 청동 조각의 정점이라 불리며, ‘부드러운 곡선

In [32]:
# 3) 테스트 import

# llm = ChatOllama(base_url="http://127.0.0.1:11434", model="gpt-oss:20b", temperature=0.3)
resp = llm.invoke([
    SystemMessage(content="너는 AI 문화유산 스마트도슨트야."),
    HumanMessage(content="장곡사 미륵불 괘불탱에 대해서 설명해줘."),
])
print(resp.content)

[GIN] 2025/09/07 - 12:24:48 | 200 |  34.54837296s |       127.0.0.1 | POST     "/api/chat"
## 장곡사 미륵불 괘불탱이란?

**장곡사(長谷寺)**는 경상북도 경주에 위치한 사찰로, 신라 시대(7세기) 초기에 세워졌습니다.  
장곡사는 특히 ‘미륵불’과 ‘괘불탱’이라는 두 가지 대표적인 문화재가 있는 곳으로 유명합니다. 이 두 유물은 모두 **국보**로 지정되어 있으며, 신라의 석조미술과 불교 문화가 얼마나 정교하고 풍부했는지를 보여주는 귀중한 예시입니다.

---

### 1. 미륵불 (미륵불상)

| 항목 | 내용 |
|------|------|
| **형태** | 2.5m 높이의 석조 미륵불상 |
| **재료** | 단일석(흑암석) |
| **시대** | 신라 7세기 초 |
| **특징** | ① **평온한 표정** – 미륵이 바라보는 눈빛이 부드럽고 온화함<br>② **복장과 장식** – 섬세하게 조각된 장식품(복장, 머리 장식, 팔찌 등)<br>③ **자연스러운 비율** – 신체 비례가 자연스럽고 균형 잡힘 |
| **의의** | 미륵보살은 ‘미래의 부처’로서, 인간이 불교적 깨달음에 이르는 길을 상징합니다. 장곡사의 미륵불은 신라 시대의 불교 예술이 얼마나 정교했는지를 보여주는 대표작입니다. |
| **문화재 등급** | 국보 제112호 |

---

### 2. 괘불탱 (Gwaebul‑tang)

> **괘불탱**은 ‘괘불(蓮佛)’이라는 ‘연꽃 불상’이 들어 있는 작은 석조 탑을 말합니다.  
> ‘괘불’은 ‘연꽃’이라는 뜻을 가지고 있어, 깨끗하고 순수한 깨달음을 상징합니다.

| 항목 | 내용 |
|------|------|
| **형태** | 1.5m 정도의 작은 석조 탑(정사형) |
| **구조** | ① **연꽃 모양의 기초** – 연꽃을 형상화한 기초부<br>② **정사형 본체** – 4개의 벽면이 직각으로 연결된 구조<br>③ **탑 꼭대기** – 작은 돔형 마루 

### ollama - gemma3:12b

In [39]:
import os, subprocess

# 원하는 경로 지정 (여기선 ./models)
os.environ["OLLAMA_MODELS"] = "./models"

# 디렉토리가 없으면 먼저 생성
os.makedirs(os.environ["OLLAMA_MODELS"], exist_ok=True)

# gemma3:12b 모델 풀
subprocess.run(["ollama", "pull", "gemma3"], check=True, env=os.environ)


[GIN] 2025/09/07 - 12:36:40 | 200 |      33.555µs |       127.0.0.1 | HEAD     "/"


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ 

[GIN] 2025/09/07 - 12:36:41 | 200 |  746.920851ms |       127.0.0.1 | POST     "/api/pull"


pulling manifest ⠦ pulling manifest 
pulling aeda25e63ebd: 100% ▕██████████████████▏ 3.3 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling dd084c7d92a3: 100% ▕██████████████████▏ 8.4 KB                         
pulling 3116c5225075: 100% ▕██████████████████▏   77 B                         
pulling b6ae5839783f: 100% ▕██████████████████▏  489 B                         
verifying sha256 digest 
writing manifest 
success 


CompletedProcess(args=['ollama', 'pull', 'gemma3'], returncode=0)

[GIN] 2025/09/07 - 12:36:52 | 200 |      26.533µs |       127.0.0.1 | HEAD     "/"
[GIN] 2025/09/07 - 12:36:52 | 200 |    6.479818ms |       127.0.0.1 | GET      "/api/tags"


In [40]:
# Ollama 연결
llm1 = ChatOllama(
    base_url="http://127.0.0.1:11434",  # ollama serve가 켜져 있는 주소
    model="gemma3:latest",
    temperature=0.7,
    num_ctx=2048,
)

In [41]:
# 단일 메시지
resp = llm1.invoke("장곡사 미륵불 괘불탱에 대해서 설명해줘.")
print(resp.content)

time=2025-09-07T12:38:21.125+09:00 level=INFO source=server.go:398 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --model models/blobs/sha256-aeda25e63ebd698fab8638ffb778e68bed908b960d39d0becc650fa981609d25 --port 43343"
time=2025-09-07T12:38:21.150+09:00 level=INFO source=runner.go:1251 msg="starting ollama engine"
time=2025-09-07T12:38:21.151+09:00 level=INFO source=runner.go:1286 msg="Server listening on 127.0.0.1:43343"
time=2025-09-07T12:38:21.409+09:00 level=INFO source=server.go:503 msg="system memory" total="1007.1 GiB" free="926.1 GiB" free_swap="8.0 GiB"
time=2025-09-07T12:38:21.411+09:00 level=INFO source=memory.go:36 msg="new model will fit in available VRAM across minimum required GPUs, loading" model=models/blobs/sha256-aeda25e63ebd698fab8638ffb778e68bed908b960d39d0becc650fa981609d25 library=cuda parallel=1 required="5.3 GiB" gpus=1
time=2025-09-07T12:38:21.413+09:00 level=INFO source=server.go:543 msg=offload library=cuda layers.requested=-1 laye

[GIN] 2025/09/07 - 12:38:35 | 200 | 15.831757181s |       127.0.0.1 | POST     "/api/chat"
장곡사 미륵불 괘불탱은 조선 후기 대표적인 불화 작품 중 하나로, 한국 불교 미술의 뛰어남을 보여주는 중요한 유물입니다. 다음은 장곡사 미륵불 괘불탱에 대한 자세한 설명입니다.

**1. 괘불탱이란?**

*   **괘(掛)**: 널브러뜨린 둥근 천을 의미합니다.
*   **불탱(佛像)**: 불상(佛像)을 그림으로 표현한 것을 의미합니다.
*   **괘불탱**은 괘를 배경으로 불상을 그린 널브러뜨린 형태의 불화입니다. 
*   일반적으로 괘불탱은 좌선(坐禪) 자세를 한 불상을 널브러뜨려 그리는 방식으로, 불교 신자들이 정제된 마음으로 불상을 바라보고 예배하는 모습을 상징합니다.

**2. 장곡사 미륵불 괘불탱의 특징**

*   **크기:** 높이 7.8m, 가로 13.8m, 세로 3.4m의 거대한 규모를 자랑합니다.
*   **화기(筆法):** 조선 후기 대표적인 화풍인 ‘청화(淸畵)’ 기법이 사용되었습니다. 깨끗하고 맑은 느낌을 주며, 섬세하고 정교한 묘사가 돋보입니다.
*   **색채:** 붉은색, 금색, 흰색 등 강렬하고 밝은 색채를 사용하여 불상의 위엄과 신성함을 강조했습니다.
*   **미륵불의 모습:** 미륵불은 현생불로서, 석가모니 이전의 현현으로 여겨졌습니다. 괘불탱의 미륵불은 수수색의 옷을 입고, 지나상(指經) 자세로, 손으로 경전을 들고 있는 모습으로 표현되어 지혜와 수행의 의미를 담고 있습니다.
*   **장식:** 괘불탱에는 다양한 장식 요소들이 사용되었습니다. 
    *   **보주(寶珠):** 미륵불의 머리에 있는 보주를 통해 지혜와 깨달음을 상징합니다.
    *   **수의(垂衣):** 미륵불의 옷을 흘러내리는 모습은 인간의 욕망을 초월하여 깨달음을 얻고자 하는 염원을 나타냅니다.
    *   **장미( garlands):** 괘의 양쪽에 장미를 드리워 화려함을 더했

In [42]:
resp = llm1.invoke([
    SystemMessage(content="너는 AI 문화유산 스마트도슨트야."),
    HumanMessage(content="장곡사 미륵불 괘불탱에 대해서 설명해줘."),
])
print(resp.content)

[GIN] 2025/09/07 - 12:39:38 | 200 |   7.44814141s |       127.0.0.1 | POST     "/api/chat"
안녕하세요. 저는 AI 문화유산 스마트도슨트입니다. 장곡사 미륵불 괘불탱에 대해 자세히 설명해 드리겠습니다. 

**장곡사 미륵불 괘불탱이란?**

장곡사 미륵불 괘불탱은 조선 후기 대표적인 불화로, 1764년에 제작된 괘상(掛像)입니다. 괘상은 널빤지 위에 괘를 덧댄 형태로, 불상을 얹어놓고 훼손을 막는 역할을 합니다. 이 괘불탱은 특히 뛰어난 화제성과 제작 기술, 그리고 역사적 의미를 지니고 있어 한국 불화의 중요한 유산으로 평가받고 있습니다.

**괘불탱의 특징**

*   **크기:** 높이 7.8m, 가로 13.7m의 거대한 크기를 자랑하며, 당시 제작 기술의 수준을 보여줍니다.
*   **화제성:** 미륵불은 석가모니 이전의 불상으로, 미래의 석가모니를 상징하는 존재입니다. 장곡사 미륵불 괘불탱은 미륵불의 신성함과 미래에 대한 희망을 담고 있어 당시 사회에 큰 영향을 미쳤습니다.
*   **제작 기술:** 괘상 제작은 매우 정교하고 숙련된 기술을 필요로 합니다. 괘상 제작에는 옻칠, 황금, 다양한 색상의 물감 등 고가의 재료가 사용되었으며, 제작 과정 또한 여러 단계의 과정을 거쳐 완성되었습니다.
*   **화제:** 괘상 위에 괘를 덧댄 형태는 불상을 보호하고 훼손을 막는 역할을 합니다. 또한, 괘상 위에 그려진 화제는 불상의 신성함을 강조하고, 불상의 모습을 더욱 돋보이게 합니다.
*   **역사적 의미:** 장곡사는 조선 초기부터 사찰로 존립해 온 곳으로, 장곡사 미륵불 괘불탱은 장곡사의 역사와 문화를 담고 있습니다.

**괘불탱의 내용**

*   **미륵불의 모습:** 괘불탱의 중심에는 미륵불이 묘형을 하고 있습니다. 미륵불은 수수께끼 같은 미소를 짓고 있으며, 손에는 서원지를 들고 있습니다. 서원지는 미륵불의 구원을 담은 inscription입니다.
*   **화려한 색채:** 